## Summary

This notebook shows how to do streaming responses with Instructor. It's also notable that this combo (LLM + Instructor) can create structured synthetic data.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

In [1]:
%%capture
!pip install instructor groq openai

On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY

GROQ_API_KEY

In [2]:
import openai
import groq
import instructor
from pydantic import BaseModel, Field
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')

Now to the cool stuff...

In [9]:
inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

class User(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    gender: str = Field(description="The gender of the user")
    city: str = Field(description="The city of the user")
    state: str = Field(description="The state of the user")
    country: str = Field(description="The country of the user")
    backstory: str = Field(description="A backstory about the user")

user_stream = client.chat.completions.create_partial(   # create_partial is what does the asynchronous streaming (usually it's "create")
    model="gpt-4-turbo" if inference_provider == "openai" else "llama3-70b-8192",
    messages=[
        {"role": "user", "content": "Create 1 synthetic user"},   # ask the LLM to make up some data
    ],
    response_model=User,
)

# below is some overly complicated code to print the results...really just a prettier version of:
# for user in user_stream:
#     print(user)

previous_output = None
for user in user_stream:
    # this loops with every token
    user_dict = user.model_dump()
    current_output = ", ".join(f"{key}: {value}" for key, value in user_dict.items() if value is not None)
    if current_output != previous_output:
        print(current_output)   # only print when there's a complete new value i.e. all tokens have arrived
        previous_output = current_output




name: Jake Stevens
name: Jake Stevens, age: 34
name: Jake Stevens, age: 34, gender: Male
name: Jake Stevens, age: 34, gender: Male, city: Austin
name: Jake Stevens, age: 34, gender: Male, city: Austin, state: Texas
name: Jake Stevens, age: 34, gender: Male, city: Austin, state: Texas, country: USA
name: Jake Stevens, age: 34, gender: Male, city: Austin, state: Texas, country: USA, backstory: Jake is a software developer living in Austin, Texas. He loves hiking and exploring the outdoors, often taking trips to the nearby Hill Country. Passionate about technology and innovation, Jake spends his spare time working on personal coding projects and contributing to open source communities.
